# Training metrics

*Metrics* for training fastai models are simply functions that take `input` and `target` tensors, and return some metric of interest for training. You can write your own metrics by defining a function of that type, and passing it to [`Learner`](/basic_train.html#Learner) in the [code]metrics[/code] parameter, or use one of the following pre-defined functions.

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai import *

## Predefined metrics:

In [ ]:
show_doc(accuracy)

<h4 id="accuracy"><code>accuracy</code><a href="https://github.com/fastai/fastai/blob/master/fastai/metrics.py#L34" class="source_link">[source]</a></h4>

> <code>accuracy</code>(`input`:`Tensor`, `targs`:`LongTensor`) → `Rank0Tensor`

Compute accuracy with `targs` when `input` is bs * n_classes.  

In [ ]:
jekyll_warn("This metric is intended for classification of objects belonging to a single class.")

<div markdown="span" class="alert alert-danger" role="alert"><i class="fa fa-danger-circle"></i> <b>Warning: </b>This metric is intended for classification of objects belonging to a single class.</div>

In [ ]:
show_doc(accuracy_thresh)

<h4 id="accuracy_thresh"><code>accuracy_thresh</code><a href="https://github.com/fastai/fastai/blob/master/fastai/metrics.py#L19" class="source_link">[source]</a></h4>

> <code>accuracy_thresh</code>(`y_pred`:`Tensor`, `y_true`:`Tensor`, `thresh`:`float`=`0.5`, `sigmoid`:`bool`=`True`) → `Rank0Tensor`

Compute accuracy when `y_pred` and `y_true` are the same size.  

Prediction are compared to `thresh` after `sigmoid` is maybe applied. Then we count the numbers that match the targets.

In [ ]:
jekyll_note("This function is intended for one-hot-encoded targets (often in a multiclassification problem).")

<div markdown="span" class="alert alert-info" role="alert"><i class="fa fa-info-circle"></i> <b>Note: </b>This function is intended for one-hot-encoded targets (often in a multiclassification problem).</div>

In [ ]:
show_doc(dice)

<h4 id="dice"><code>dice</code><a href="https://github.com/fastai/fastai/blob/master/fastai/metrics.py#L24" class="source_link">[source]</a></h4>

> <code>dice</code>(`input`:`FloatTensor`, `targs`:`LongTensor`, `iou`:`bool`=`False`) → `Rank0Tensor`

Dice coefficient metric for binary target. If `iou=True`, return iou metric.  

In [ ]:
show_doc(error_rate)

<h4 id="error_rate"><code>error_rate</code><a href="https://github.com/fastai/fastai/blob/master/fastai/metrics.py#L41" class="source_link">[source]</a></h4>

> <code>error_rate</code>(`input`:`Tensor`, `targs`:`Tensor`) → `Rank0Tensor`

1 - [`accuracy`](/metrics.html#accuracy)  

In [ ]:
show_doc(exp_rmspe)

<h4 id="exp_rmspe"><code>exp_rmspe</code><a href="https://github.com/fastai/fastai/blob/master/fastai/metrics.py#L45" class="source_link">[source]</a></h4>

> <code>exp_rmspe</code>(`pred`:`FloatTensor`, `targ`:`FloatTensor`) → `Rank0Tensor`

Exp RMSE between `pred` and `targ`.  

In [ ]:
show_doc(fbeta)

<h4 id="fbeta"><code>fbeta</code><a href="https://github.com/fastai/fastai/blob/master/fastai/metrics.py#L7" class="source_link">[source]</a></h4>

> <code>fbeta</code>(`y_pred`:`Tensor`, `y_true`:`Tensor`, `thresh`:`float`=`0.2`, `beta`:`float`=`2`, `eps`:`float`=`1e-09`, `sigmoid`:`bool`=`True`) → `Rank0Tensor`

Computes the f_beta between `y_pred` and `y_true` in a multi-classification task.  

`beta` determines the value of the fbeta applied, `eps` is there for numeric stability. If `sigmoid=True`, a sigmoid is applied to the predictions before comparing them to `thresh` then to the targets. See the [F1 score wikipedia page](https://en.wikipedia.org/wiki/F1_score) for details on the fbeta score.

In [ ]:
jekyll_note("This function is intended for one-hot-encoded targets (often in a multiclassification problem).")

<div markdown="span" class="alert alert-info" role="alert"><i class="fa fa-info-circle"></i> <b>Note: </b>This function is intended for one-hot-encoded targets (often in a multiclassification problem).</div>

In [ ]:
show_doc(Fbeta_binary, title_level=3)

<h3 id="Fbeta_binary"><code>class</code> <code>Fbeta_binary</code><a href="https://github.com/fastai/fastai/blob/master/fastai/metrics.py#L54" class="source_link">[source]</a></h3>

> <code>Fbeta_binary</code>(`beta2`:`int`=`2`, `eps`:`float`=`1e-09`, `clas`:`int`=`1`) :: [`Callback`](/callback.html#Callback)

Computes the fbeta between preds and targets for single-classification.  

## Creating your own metric

Creating a new metric can be as simple as creating a new function. If you metric is an average over the total number of elements in your dataset, just write the function that will compute it on a batch (taking `pred` and `targ` as arguments). It will then be automatically averaged over the batches (taking their different sizes into acount).

Sometimes metrics aren't simple averages however. If we take the example of precision for instance, we have to divide the number of true positives by the number of predictions we made for that class. This isn't an average over the number of elements we have in the dataset, we only consider those where we made a positive prediction for a specific thing. Computing the precision for each batch, then averaging them will yield to a result that may be close to the real value, but won't be it exactly (and it really depends on how you deal with special case of 0 positive predicitions).

This why in fastai, every metric is implemented as a callback. If you pass a regular function, the library transforms it to a proper callback called `AverageCallback`. The callback metrics are only called during the validation phase, and only for the following events: 
- <code>on_epoch_begin</code> (for initialization)
- <code>on_batch_begin</code> (if we need to have a look at the input/target and maybe modify them)
- <code>on_batch_end</code> (to analyze the last results and update our computation)
- <code>on_epoch_end</code>(to wrap up the final result that should be stored in `.metric`)

As an example, is here the exact implementation of the [`AverageMetric`](/callback.html#AverageMetric) callback that transforms a function like [`accuracy`](/metrics.html#accuracy) into a metric callback.

In [ ]:
class AverageMetric(Callback):
    def __init__(self, func):
        self.func, self.name = func, func.__name__

    def on_epoch_begin(self, **kwargs):
        self.val, self.count = 0.,0

    def on_batch_end(self, last_output, last_target, train, **kwargs):
        self.count += last_target.size(0)
        self.val += last_target.size(0) * self.func(last_output, last_target).detach().item()

    def on_epoch_end(self, **kwargs):
        self.metric = self.val/self.count

And here is another example that properly computes the precision for a given class.

In [ ]:
class Precision(Callback):
    
    def on_epoch_begin(self, **kwargs):
        self.correct, self.total = 0, 0
    
    def on_batch_end(self, last_output, last_target, **kwargs):
        preds = last_output.argmax(1)
        self.correct += ((preds==0) * (last_target==0)).float().sum()
        self.total += (preds==0).float().sum()
    
    def on_epoch_end(self, **kwargs):
        self.metric = self.correct/self.total

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(Fbeta_binary.on_batch_end)

<h4 id="Fbeta_binary.on_batch_end"><code>on_batch_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/metrics.py#L65" class="source_link">[source]</a></h4>

> <code>on_batch_end</code>(`last_output`, `last_target`, `kwargs`)

Called at the end of the batch.  

In [ ]:
show_doc(Fbeta_binary.on_epoch_begin)

<h4 id="Fbeta_binary.on_epoch_begin"><code>on_epoch_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/metrics.py#L60" class="source_link">[source]</a></h4>

> <code>on_epoch_begin</code>(`kwargs`)

At the beginning of each epoch.  

In [ ]:
show_doc(Fbeta_binary.on_epoch_end)

<h4 id="Fbeta_binary.on_epoch_end"><code>on_epoch_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/metrics.py#L73" class="source_link">[source]</a></h4>

> <code>on_epoch_end</code>(`kwargs`)

Called at the end of an epoch.  

## New Methods - Please document or move to the undocumented section